In [ ]:
!pip install --upgrade tensorflow
!pip install tensorflow-datasets

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load data into pandas
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "text"])
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "text"])

# Convert labels to 0 (ham) and 1 (spam)
train_data['label'] = train_data['label'].map({'ham':0, 'spam':1})
test_data['label'] = test_data['label'].map({'ham':0, 'spam':1})

print(train_data.head())


In [ ]:
# Tokenize the text
vocab_size = 5000       # increase vocab to cover more words
embedding_dim = 32      # larger embedding
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['text'])

max_length = 50
trunc_type = 'post'
padding_type = 'post'

train_sequences = tokenizer.texts_to_sequences(train_data['text'])
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_data['text'])
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])


In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

num_epochs = 15
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels))


In [ ]:
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    pred_prob = float(model.predict(padded)[0][0])
    label = 'spam' if pred_prob > 0.5 else 'ham'
    return [pred_prob, label]

# Example test
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(f"Message: {msg}\nPredicted: {prediction}\nExpected: {ans}\n")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()
